In [39]:
#import dependencies 
import pandas as pd
from bs4 import BeautifulSoup as bs
import pymongo
from splinter import Browser
import requests
from selenium import webdriver

In [40]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [41]:
#scrape for Nasa News
url="https://mars.nasa.gov/news/"
browser.visit(url)
html=browser.html
soup=bs(html, 'html.parser')

In [42]:
#collect the latest News Title and Paragraph Text 
news_title= soup.find('div', class_='content_title').get_text()
news_ptext= soup.find('div', class_='article_teaser_body').get_text()
print(news_title)
print(news_ptext)

Mars Now
NASA released an independent review report Tuesday indicating the agency is well positioned for its Mars Sample Return campaign to bring pristine samples from Mars to Earth for scientific study.


In [43]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html=browser.html
soup=bs(html,"html.parser")

In [44]:
image=soup.find('img', class_="thumb")["src"]
featured_image_url= 'https://jpl.nasa.gov'+ image
print(featured_image_url)

https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA24242-640x350.jpg


In [45]:
#Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
url='https://space-facts.com/mars/'
facts_table=pd.read_html(url)
facts_df=facts_table[0]
facts_df= facts_df.rename(columns={0:'Mars Planet Profile', 1:'Measurement'})
facts_df=facts_df.set_index('Mars Planet Profile')
facts_df

,Measurement
Mars Planet Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [46]:
#Convert to html table string
facts_html=facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Measurement</th>
    </tr>
    <tr>
      <th>Mars Planet Profile</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [57]:
#Obtain high resolution images for each of Mar's hemisheres.
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
base_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(base_url)
html=browser.html
soup=bs(html,"html.parser")

In [58]:
titles=soup.find_all('h3')
for title in titles:
    browser.links.find_by_partial_text("Hemisphere")
    
print(titles)

[<h3>Cerberus Hemisphere Enhanced</h3>, <h3>Schiaparelli Hemisphere Enhanced</h3>, <h3>Syrtis Major Hemisphere Enhanced</h3>, <h3>Valles Marineris Hemisphere Enhanced</h3>]


In [60]:
hem_image_urls=[]
hem_dictionary={'title': [],'img_url': []}
titles=soup.find_all('h3')
for i in titles:
    long_title=i.get_text()
    title=long_title.strip('Enhanced')
    browser.click_link_by_partial_text(long_title)
    url=browser.find_link_by_partial_href('download')['href']
    hem_dictionary={'title': title,'img_url': url}
    hem_image_urls.append(hem_dictionary)
    browser.visit(base_url)
print(hem_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
